In [186]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn import tree

Carregando a base de dados.

In [187]:
url = 'dados_ibovespa.csv'
df = pd.read_csv(url)

Agora iremos fazer uma breve análise na nossa base de dados.

In [188]:
df.isnull().sum()

Data        0
Último      0
Abertura    0
Máxima      0
Mínima      0
Vol.        1
Var%        0
dtype: int64

Verificamos se há algum dado nulo na nossa base, e podemos perceber que a base esta completinha.

In [189]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4953 entries, 0 to 4952
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Data      4953 non-null   object 
 1   Último    4953 non-null   float64
 2   Abertura  4953 non-null   float64
 3   Máxima    4953 non-null   float64
 4   Mínima    4953 non-null   float64
 5   Vol.      4952 non-null   object 
 6   Var%      4953 non-null   object 
dtypes: float64(4), object(3)
memory usage: 271.0+ KB


Aqui notamos que o nosso campo de "Data" esta como string, sendo assim precisamos converter ela para que possamos usar ela como index.

In [190]:
df["Data"] = pd.to_datetime(df["Data"], dayfirst=True)

In [191]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4953 entries, 0 to 4952
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Data      4953 non-null   datetime64[ns]
 1   Último    4953 non-null   float64       
 2   Abertura  4953 non-null   float64       
 3   Máxima    4953 non-null   float64       
 4   Mínima    4953 non-null   float64       
 5   Vol.      4952 non-null   object        
 6   Var%      4953 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 271.0+ KB


In [192]:
df.size

34671

Agora iremos ordenar nossos dados dos mais recentes ao mais antigos, para que a análise fique mais fácil.

In [193]:
df = df.sort_values("Data").reset_index(drop=True)

In [194]:
df.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,2005-07-11,25.016,24.425,25.032,24.425,"100,42M","2,43%"
1,2005-07-12,25.536,25.027,25.576,24.932,"94,91M","2,08%"
2,2005-07-13,25.856,25.557,26.043,25.557,"107,28M","1,25%"
3,2005-07-14,25.920,25.859,26.142,25.725,"113,35M","0,25%"
4,2005-07-15,25.222,25.916,25.916,25.222,"72,88M","-2,69%"


Agora iremos selecionar apenas as colunas que façam sentidos no nosso contexto.

In [195]:
df = df[["Data", "Último"]]

Quando utilizamos modelos, é necessário seguir um padrão de nomenclatura, sendo assim, iremos renomar os nomes das nossas colunas.

In [196]:
df = df.rename(columns={"Data": "ds", "Último": "y"})

In [197]:
df.head()

,ds,y
0,2005-07-11,25.016
1,2005-07-12,25.536
2,2005-07-13,25.856
3,2005-07-14,25.920
4,2005-07-15,25.222


Agora iremos criar a nossa variável target que irá receber 1 para subiu e 0 para desceu, para que futuramente podermos análisar o resultado.

In [198]:
df["target"] = (df["y"].shift(-1) > df["y"]).astype(int)
df = df.dropna().reset_index(drop=True)

In [199]:
df.head()

,ds,y,target
0,2005-07-11,25.016,1
1,2005-07-12,25.536,1
2,2005-07-13,25.856,1
3,2005-07-14,25.920,0
4,2005-07-15,25.222,1


Agora iremos criar algumas variáveis que servirão de auxilio para o nosso modelo entender um pouco mais sobre o motivo das possiveis variações dos resultados.

In [ ]:
df["ret_1d"] = df["y"].pct_change() * 100 
df["ret_3d"] = df["y"].pct_change(3) * 100
df["ret_5d"] = df["y"].pct_change(5) * 100

df["sma_3"] = df["y"].rolling(3).mean()
df["sma_7"] = df["y"].rolling(7).mean()
df["sma_diff"] = df["y"] - df["sma_3"]

df["lag_1"] = df["y"].shift(1)
df["lag_2"] = df["y"].shift(2)

In [201]:
df.fillna(df.mean(), inplace=True)

In [202]:
df.head()

,ds,y,target,ret_1d,ret_3d,ret_5d,sma_3,sma_7,sma_diff,lag_1,lag_2
0,2005-07-11,25.016,1,0.048144,0.141110,0.232605,74.909213,74.902906,0.022768,74.899295,74.886287
1,2005-07-12,25.536,1,2.078670,0.141110,0.232605,74.909213,74.902906,0.022768,25.016000,74.886287
2,2005-07-13,25.856,1,1.253133,0.141110,0.232605,25.469333,74.902906,0.386667,25.536000,25.016000
3,2005-07-14,25.920,0,0.247525,3.613687,0.232605,25.770667,74.902906,0.149333,25.856000,25.536000
4,2005-07-15,25.222,1,-2.692901,-1.229637,0.232605,25.666000,74.902906,-0.444000,25.920000,25.856000


Agora iremos separar o que será nossa variável target e as que serão nossas variáveis características.

In [203]:
x = df[["ret_1d", "ret_3d", "ret_5d", "sma_3", "sma_7", "sma_diff", "lag_1", "lag_2"]]
y = df["target"]

Agora iremos separar a nossa base em treino e teste

In [204]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, stratify=y, random_state=7)

Agora iremos instanciar nosso modelo.

In [205]:
model = DecisionTreeClassifier(random_state=7, criterion='entropy', max_depth = 2)

Agora iremos treinar nosso modelo, de acordo com o que separamos anteriormente.

In [206]:
model.fit(x_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=2, random_state=7)

In [207]:
y_pred = model.predict(x_test)

In [208]:
accuracy_score(y_test, y_pred)


0.5201612903225806